## **Challenge 33: Implement Diffie-Hellman**

In [2]:
import secrets
# 
import src.dh as dh
import src.sha1 as sha1
import src.aes128 as aes128
import src.sha256 as sha256
import src.convert as convert

In [349]:
prime = 37
generator = 5
print(f"Diffie-Hellman with parameters: \n- Prime: {prime}\n- Generator: {generator}\n")

# Generate alice and bob's private and public keys
private_key_a, public_key_a = dh.gen_keypair(prime, generator)
private_key_b, public_key_b = dh.gen_keypair(prime, generator)
print(f"Alice's Private Key: {private_key_a}, Public Key: {public_key_a}")
print(f"Bob's Private Key: {private_key_b}, Public Key: {public_key_b}\n")

# Generate alice and bob's shared secrets (should be the same)
secret_a = dh.shared_secret(prime, prime, private_key_b)
secret_b = dh.shared_secret(prime, prime, private_key_a)
print(f"Alice's Secret: {secret_a}")
print(f"Bob's Secret: {secret_b}")

Diffie-Hellman with parameters: 
- Prime: 37
- Generator: 5

Alice's Private Key: 7, Public Key: 18
Bob's Private Key: 25, Public Key: 19

Alice's Secret: 24
Bob's Secret: 24


In [4]:
prime = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
generator = 2
print(f"Diffie-Hellman with parameters: \n- Prime: {prime}\n- Generator: {generator}\n")

# Generate alice and bob's private and public keys
private_key_a, public_key_a = dh.gen_keypair(prime, generator)
private_key_b, public_key_b = dh.gen_keypair(prime, generator)
print(f"Alice's Private Key: {private_key_a}, Public Key: {public_key_a}")
print(f"Bob's Private Key: {private_key_b}, Public Key: {public_key_b}\n")

# Generate alice and bob's shared secrets (should be the same)
secret_a = dh.shared_secret(prime, public_key_a, private_key_b)
secret_b = dh.shared_secret(prime, public_key_b, private_key_a)
print(f"Alice's Secret: {secret_a}")
print(f"Bob's Secret: {secret_b}")

Diffie-Hellman with parameters: 
- Prime: 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
- Generator: 2

Alice's Private Key: 1431037398876195351979475207254443037390790472903756546558393589510023445962842429953588711804183995326582534418922377116723698468822327445583691579761143246514589024296833386214377002831129006264269117825032022566425753808807146946191673465870254957957368755203426953916164411080842749289925461953413765957364346115461286212447266781414450188561163214789481867767617206869541893861477090000948908616096279744951175611059580233866243465591795763095318841348

In [5]:
# Generate a 128 bit key from the hash of the shared secret
key = sha256.digest(convert.bigint2bytes(secret_a))[:16]
print(f"\n128 bit key: {key} ({len(key)} bytes)")


128 bit key: b'\\\x00\xe1b#G\xca\xaa\xc0\x80\x0cB\x8e\x8b<\x8e' (16 bytes)


## **Challenge 34: Implement a MITM key-fixing attack on Diffie-Hellman with parameter injection**

## Normal Protocol
```
(1) A->B
    Send "p", "g", "A"
(2) B->A
    Send "B"
(3) A->B
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv
(4) B->A
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv 
```

In [7]:
# (1) Send "p", "g", "A" to Bob (A->B)
a_p = 37
a_g = 5
a_priv, a_pub = dh.gen_keypair(a_p, a_g)

# Bob receives "p", "g", "A" 
b_p = a_p
b_g = a_g
b_A = a_pub
b_priv, b_pub = dh.gen_keypair(b_p, b_g)

# (2) Send "B" to Alice (B->A)
a_B = b_pub

# Compute shared secret independently
a_s = dh.shared_secret(a_p, a_B, a_priv)
b_s = dh.shared_secret(b_p, b_A, b_priv)

# Derive AES key from shared secret
a_key = sha1.digest(convert.bigint2bytes(a_s))[:16]
b_key = sha1.digest(convert.bigint2bytes(b_s))[:16]

# (3) Send message to Bob over encrypted channel (A->B)
a_iv = secrets.token_bytes(16)
a_msg = b"Hello Bob!!!!!!!"
a_ciphertext = aes128.cbc_encrypt(a_msg, a_key, a_iv)

# Receive message from Alice and decrypt (using Bob's AES key)
b_plaintext = aes128.cbc_decrypt(a_ciphertext, b_key, a_iv)
print(f"Bob received: {b_plaintext}")

# (4) Echo message to Alice over encrypted channel (A->B)
b_iv = secrets.token_bytes(16)
b_ciphertext = aes128.cbc_encrypt(b_plaintext, b_key, b_iv)

# Receive message from Bob and decrypt (using Alice's AES key)
a_plaintext = aes128.cbc_decrypt(b_ciphertext, a_key, b_iv)
print(f"Alice received: {a_plaintext}")

Bob received: b'Hello Bob!!!!!!!'
Alice received: b'Hello Bob!!!!!!!'


## Man in the Middle Protocol

```
(1) A->M
    Send "p", "g", "A"
(2) M->B
    Send "p", "g", "p"
(3) B->M
    Send "B"
(4) M->A
    Send "p"
(5) A->M
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv
(6) M->B
    Relay that to B
(7) B->M
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv
(8) M->A
    Relay that to A
```

In [285]:
# (1) Send "p", "g", "A" to Bob (A->M)
a_p = 37
a_g = 5
a_priv, a_pub = dh.gen_keypair(a_p, a_g)

# (2) Send "p", "g", "p" to Bob (M->B)
b_p = a_p
b_g = a_g
b_A = a_p # Middleman intercepts and replaces A with p
b_priv, b_pub = dh.gen_keypair(b_p, b_g)
print(f"Bob's Public Key: {b_pub}, private key: {b_priv}")

# (3) Send "B" to Middleman (B->M)
m_B = b_pub

# (4) Send "p" to Alice (M->A)
a_B = a_p

# Compute shared secret independently
a_s = dh.shared_secret(a_p, a_B, a_priv)
b_s = dh.shared_secret(b_p, b_A, b_priv)
print(f"Alice's Secret == Bob's Secret == {a_s}")

# Derive AES key from shared secret
a_key = sha1.digest(convert.bigint2bytes(a_s))[:16]
b_key = sha1.digest(convert.bigint2bytes(b_s))[:16]

# (5) Send message to Bob over encrypted channel (A->M) 
a_iv = secrets.token_bytes(16)
a_msg = b"Hello Bob!!!!!!!"
a_ciphertext = aes128.cbc_encrypt(a_msg, a_key, a_iv)

# (6) Send message to Alice over encrypted channel (M->B)
m_ciphertext = a_ciphertext
m_key = sha1.digest(16 * b"\x00")[:16]
m_plaintext = aes128.cbc_decrypt(m_ciphertext, m_key, a_iv)
print(f"Middleman received: {m_plaintext} (encrypted with Alice's key)")

# Receive message from Alice and decrypt (using Bob's AES key)
b_plaintext = aes128.cbc_decrypt(m_ciphertext, b_key, a_iv)
print(f"Bob received: {b_plaintext}")

# (5) Echo message to Alice over encrypted channel (A -> B)
b_iv = secrets.token_bytes(16)
b_ciphertext = aes128.cbc_encrypt(b_plaintext, b_key, b_iv)

# Receive message from Bob and decrypt (using Alice's AES key)
a_plaintext = aes128.cbc_decrypt(b_ciphertext, a_key, b_iv)
print(f"Alice received: {a_plaintext}")

Bob's Public Key: 13, private key: 13
Alice's Secret == Bob's Secret == 0
Middleman received: b'\xb6\xc5\x11\\\xe9\xe01:\xfbw\xe3e\xe6}2\xbf' (encrypted with Alice's key)
Bob received: b'Hello Bob!!!!!!!'
Alice received: b'Hello Bob!!!!!!!'
